# Scoring your trained model

In the cell below, please load your model into `model`. Also if you used an image size for your input images that *isn't* 224x224, you'll need to set `image_size` to the size you used. The scoring code assumes square input images.

For example, this is how I loaded in my checkpoint:

```python
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models

class FFClassifier(nn.Module):
    
    def __init__(self, in_features, hidden_features, 
                       out_features, drop_prob=0.1):
        super().__init__()
        
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(p=drop_prob)
        
    def forward(self, x):
        x = self.drop(F.relu(self.fc1(x)))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

    
def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    
    model = models.vgg16(pretrained=False)
    for param in model.parameters():
        param.requires_grad = False

    # Put the classifier on the pretrained network
    model.classifier = FFClassifier(25088, checkpoint['hidden'], 102)
    
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

model = load_checkpoint('/home/workspace/classifier.pt')
```

Your exact code here will depend on how you defined your network in the project. Make sure you use the absolute path to your checkpoint which should have been uploaded to the `/home/workspace` directory.

Run the cell, then after loading the data, press "Test Code" below. This can take a few minutes or more depending on the size of your network. Your model needs  to reach **at least 20% accuracy** on the test set to be recorded.

In [1]:
!pip install gdown==3.6.0


  Running setup.py bdist_wheel for gdown ... done
  Stored in directory: /root/.cache/pip/wheels/97/90/fa/25654eb65da3e6da7752db71a164e0eb8f7a6fb4335eeb46ab
Successfully built gdown
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
my_file_id ="1ysfgj8ccTyESLJvK913R2u475mD4luma"

In [4]:
!gdown https://drive.google.com/uc?id={my_file_id}

Downloading...
From: https://drive.google.com/uc?id=1ysfgj8ccTyESLJvK913R2u475mD4luma
To: /home/workspace/classifier.pth
493MB [00:05, 92.3MB/s] 


In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image
from collections import OrderedDict

import time
import json
import copy

import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms

classifier = nn.Sequential(OrderedDict([
                        ('fc1', nn.Linear(25088, 4096)),
                        ('relu', nn.ReLU()),
                        ('fc2', nn.Linear(4096, 102)),
                        ('output', nn.LogSoftmax(dim=1))
                        ]))


def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)

    model = models.vgg16(pretrained=False)
    for param in model.parameters():
        param.requires_grad = False
    
    model.classifier = classifier

    # Put the classifier on the pretrained network
    model.classifier = classifier(25088, 4096, 102)

    model.load_state_dict(torch.load(checkpoint_path, map_location='cpu')['state_dict'], strict=False)

    return model



In [2]:
# Load your model to this variable
model = load_checkpoint('classifier.pth')
   
# If you used something other than 224x224 cropped images, set the correct size here
image_size = 224
# Values you used for normalizing the images. Default here are for 
# pretrained models from torchvision.
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

TypeError: forward() takes 2 positional arguments but 4 were given